### Challenge 1: Total Spend Per Customer

Calculate total amount spent by each customer.


In [0]:

from pyspark.sql import Row
from pyspark.sql.functions import (col, sum as sparkSum)

In [0]:
data = [
    Row(customer_id=1, amount=250),
    Row(customer_id=2, amount=450),
    Row(customer_id=1, amount=100),
    Row(customer_id=3, amount=300),
    Row(customer_id=2, amount=150)
]
df = spark.createDataFrame(data)
df.show()


+-----------+------+
|customer_id|amount|
+-----------+------+
|          1|   250|
|          2|   450|
|          1|   100|
|          3|   300|
|          2|   150|
+-----------+------+



In [0]:
df_total_per_customer = df.groupBy("customer_id").agg(sparkSum(col("amount")).alias("TotalAmountSpent"))
display(df_total_per_customer)

customer_id,TotalAmountSpent
1,350
2,600
3,300


Basically, I used the aggregate sum function to find total amount spent for each customer. I used import alias sparkSum in order to prevent collision with python's native sum() function.

### Challenge 2: Highest Transaction Per Day

Find the highest transaction amount for each day.


In [0]:

from pyspark.sql import Row
from pyspark.sql.functions import (col, sum as sparkSum, row_number)
from pyspark.sql.window import Window

data = [
    Row(date='2023-01-01', amount=100),
    Row(date='2023-01-01', amount=300),
    Row(date='2023-01-02', amount=150),
    Row(date='2023-01-02', amount=200)
]
df = spark.createDataFrame(data)
df.show()


+----------+------+
|      date|amount|
+----------+------+
|2023-01-01|   100|
|2023-01-01|   300|
|2023-01-02|   150|
|2023-01-02|   200|
+----------+------+



In [0]:
windowSpec = Window.partitionBy(df.date).orderBy(df.amount.desc())
df_ranked = df.withColumn("dailyRank",row_number().over(windowSpec))
df_highest = df_ranked.filter(col("dailyRank") == 1) \
    .select([col('date'), col('amount')])
df_highest.show()

+----------+------+
|      date|amount|
+----------+------+
|2023-01-01|   300|
|2023-01-02|   200|
+----------+------+



Here, since we have to find highest transaction per day, the first step would be to re-arrange amount into descending order for each day. This can be achieved with window function rank_number with partition over the date column. After this, we just select each the rows having rank of 1.

### Challenge 3: Fill Missing Cities With Default

Replace null city values with 'Unknown'.


In [0]:

from pyspark.sql import Row

data = [
    Row(customer_id=1, city='Dallas'),
    Row(customer_id=2, city=None),
    Row(customer_id=3, city='Austin'),
    Row(customer_id=4, city=None)
]
df = spark.createDataFrame(data)
df.show()

+-----------+------+
|customer_id|  city|
+-----------+------+
|          1|Dallas|
|          2|  NULL|
|          3|Austin|
|          4|  NULL|
+-----------+------+



In [0]:
df_filled = df.fillna({"city":"Unknown"})
df_filled.show()

+-----------+-------+
|customer_id|   city|
+-----------+-------+
|          1| Dallas|
|          2|Unknown|
|          3| Austin|
|          4|Unknown|
+-----------+-------+



I used fillna() to replace any null values in city column with "Unknown" value

### Challenge 4: Compute Running Total by Customer

Use a window function to compute cumulative sum of purchases per customer.


In [0]:

from pyspark.sql import Row
from pyspark.sql.functions import (lag)

data = [
    Row(customer_id=1, date='2023-01-01', amount=100),
    Row(customer_id=1, date='2023-01-02', amount=200),
    Row(customer_id=1, date='2023-01-03', amount=300),
    Row(customer_id=2, date='2023-01-01', amount=300),
    Row(customer_id=2, date='2023-01-02', amount=400),
    Row(customer_id=2, date='2023-01-04', amount=500)
]
df = spark.createDataFrame(data)
df.show()


+-----------+----------+------+
|customer_id|      date|amount|
+-----------+----------+------+
|          1|2023-01-01|   100|
|          1|2023-01-02|   200|
|          1|2023-01-03|   300|
|          2|2023-01-01|   300|
|          2|2023-01-02|   400|
|          2|2023-01-04|   500|
+-----------+----------+------+



In [0]:
windowSpec = Window.partitionBy(col("customer_id")).orderBy(col("date"))
df_check_lag = df.withColumn("previousVal",lag(col('amount'),1,0).over(windowSpec))
display(df_check_lag)

customer_id,date,amount,previousVal
1,2023-01-01,100,0
1,2023-01-02,200,100
1,2023-01-03,300,200
2,2023-01-01,300,0
2,2023-01-02,400,300
2,2023-01-04,500,400


In [0]:
#df_running_total = df_check_lag.withColumn("Running Total", col('amount')+col('previousVal'))
df_running_total = df_check_lag.withColumn("Running Total", sparkSum(col('amount')) \
    .over(windowSpec.rowsBetween(Window.unboundedPreceding, Window.currentRow)))
display(df_running_total)

customer_id,date,amount,previousVal,Running Total
1,2023-01-01,100,0,100
1,2023-01-02,200,100,300
1,2023-01-03,300,200,600
2,2023-01-01,300,0,300
2,2023-01-02,400,300,700
2,2023-01-04,500,400,1200


This was a new challenge in terms of window function. My first thought was to use lag function to get previous value and calculate sum on the go, but then i realised this would only give me sum of 2 rows max. Then on searching pyspark docs (https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.Window.html), Window function has attribute rowsBetween() where i can specify range from unboundedPreceding (which is a very large negative number, so i presume if goes back to rows before current row context) and currentRow, thus giving the running total. Another similar attribute rangeBetween() was also there, but it seems to work on the column value of the row context and not physical position of the row. So, using the sum over the window specification provided the running total. 

### Challenge 5: Average Sales Per Product

Find average amount per product.


In [0]:

from pyspark.sql import Row
from pyspark.sql.functions import (avg as sparkAvg)

data = [
    Row(product='A', amount=100),
    Row(product='B', amount=200),
    Row(product='A', amount=300),
    Row(product='B', amount=400)
]
df = spark.createDataFrame(data)
df.show()


+-------+------+
|product|amount|
+-------+------+
|      A|   100|
|      B|   200|
|      A|   300|
|      B|   400|
+-------+------+



In [0]:
data_avg_product = df.groupBy(col("product")) \
    .agg(sparkAvg(col("amount")).alias("average_amount"))
data_avg_product.show()

+-------+--------------+
|product|average_amount|
+-------+--------------+
|      A|         200.0|
|      B|         300.0|
+-------+--------------+



Since the goal is to find average per product, first we group the data by product name and apply the aggregate function avg() on the amount column. Again, sparkAvg is used as alias to avg to avoid conflicts.

### Challenge 6: Extract Year From Date

Add a column to extract year from given date.


In [0]:

from pyspark.sql import Row
from pyspark.sql.functions import (year as sparkYear)

data = [
    Row(customer='John', transaction_date='2022-11-01'),
    Row(customer='Alice', transaction_date='2023-01-01')
]
df = spark.createDataFrame(data)
df.show()


+--------+----------------+
|customer|transaction_date|
+--------+----------------+
|    John|      2022-11-01|
|   Alice|      2023-01-01|
+--------+----------------+



In [0]:
df_extract = df.withColumn("Year", sparkYear(col("transaction_date")))
df_extract.show()

+--------+----------------+----+
|customer|transaction_date|Year|
+--------+----------------+----+
|    John|      2022-11-01|2022|
|   Alice|      2023-01-01|2023|
+--------+----------------+----+



A simple year() function, like in sql, can extract year from the date column

### Challenge 7: Join Product and Sales Data

Join two DataFrames on product_id to get product names with amounts.


In [0]:

from pyspark.sql import Row

products = [
    Row(product_id=1, product_name='Phone'),
    Row(product_id=2, product_name='Tablet')
]
sales = [
    Row(product_id=1, amount=500),
    Row(product_id=2, amount=800),
    Row(product_id=1, amount=200)
]
df_products = spark.createDataFrame(products)
df_sales = spark.createDataFrame(sales)
df_products.show()
df_sales.show()


+----------+------------+
|product_id|product_name|
+----------+------------+
|         1|       Phone|
|         2|      Tablet|
+----------+------------+

+----------+------+
|product_id|amount|
+----------+------+
|         1|   500|
|         2|   800|
|         1|   200|
+----------+------+



In [0]:
df_joined = df_products.join(df_sales, df_products.product_id == df_sales.product_id, how="inner") \
    .select([df_products['*'], df_sales.amount])
display(df_joined)

product_id,product_name,amount
1,Phone,200
1,Phone,500
2,Tablet,800


#### Explanation:
I used the inner join to show product name and sales amount in a single table. Also, I used select function to select all columns of product table and just amount from sales in order to hide duplicate product_id column due to the resulting join.

### Challenge 8: Split Tags Into Rows

Given a list of comma-separated tags, explode them into individual rows.


In [0]:

from pyspark.sql import Row
from pyspark.sql.functions import (split, explode)

data = [
    Row(id=1, tags='tech,news'),
    Row(id=2, tags='sports,music'),
    Row(id=3, tags='food')
]
df = spark.createDataFrame(data)
df.show()


+---+------------+
| id|        tags|
+---+------------+
|  1|   tech,news|
|  2|sports,music|
|  3|        food|
+---+------------+



In [0]:
df_split = df.select([col('id'), split(col('tags'),',').alias("split_tags")])
df_split.show()
print(df_split.count())

+---+---------------+
| id|     split_tags|
+---+---------------+
|  1|   [tech, news]|
|  2|[sports, music]|
|  3|         [food]|
+---+---------------+

3


In [0]:
df_exploded = df_split.withColumn("individual_tag",explode(col("split_tags")))
#df_exploded.show()
df_final = df_exploded.drop(col("split_tags"))
df_final.show()

+---+--------------+
| id|individual_tag|
+---+--------------+
|  1|          tech|
|  1|          news|
|  2|        sports|
|  2|         music|
|  3|          food|
+---+--------------+



I searched for function to split the string separated by ",". This created a list of tags in the "split_tags" column but the records were still in the same row. So, based on instructions given, I again searched for "explode" function which was specifically used to generate new rows in dataframe based on array values in column provided in the argument.

### Challenge 9: Top-N Records Per Group

For each category, return top 2 records based on score.


In [0]:

from pyspark.sql import Row

data = [
    Row(category='A', name='x', score=80),
    Row(category='A', name='y', score=90),
    Row(category='A', name='z', score=70),
    Row(category='B', name='p', score=60),
    Row(category='B', name='q', score=85)
]
df = spark.createDataFrame(data)
df.show()


+--------+----+-----+
|category|name|score|
+--------+----+-----+
|       A|   x|   80|
|       A|   y|   90|
|       A|   z|   70|
|       B|   p|   60|
|       B|   q|   85|
+--------+----+-----+



In [0]:
windowSpec = Window.partitionBy(col('category')).orderBy(col('score').desc())
df_ranked = df.withColumn("rankedScore", row_number().over(windowSpec))
df_ranked.show()

+--------+----+-----+-----------+
|category|name|score|rankedScore|
+--------+----+-----+-----------+
|       A|   y|   90|          1|
|       A|   x|   80|          2|
|       A|   z|   70|          3|
|       B|   q|   85|          1|
|       B|   p|   60|          2|
+--------+----+-----+-----------+



In [0]:
df_top2 = df_ranked.filter(col("rankedScore") <= 2).select([
    col("category"), col("name"), col("score")
])
df_top2.show()

+--------+----+-----+
|category|name|score|
+--------+----+-----+
|       A|   y|   90|
|       A|   x|   80|
|       B|   q|   85|
|       B|   p|   60|
+--------+----+-----+



This was similar to **Challenge 2** but instead of just getting highest(top 1), we need to get top 2 for each category. So, a ranking function (row_number()) follwed by filter for rank column <= 2 would give the top 2 records for each category

### Challenge 10: Null Safe Join

Join two datasets where join key might have nulls, handle using null-safe join.


In [0]:

from pyspark.sql import Row

data1 = [
    Row(id=1, name='John'),
    Row(id=None, name='Mike'),
    Row(id=2, name='Alice'),
    Row(id=None, name='Johnson'),
]
data2 = [
    Row(id=1, salary=5000),
    Row(id=None, salary=3000),
    Row(id=None, salary=2000),
    Row(id=None, salary=1500)
]
df1 = spark.createDataFrame(data1)
df2 = spark.createDataFrame(data2)
df1.show()
df2.show()


+----+-------+
|  id|   name|
+----+-------+
|   1|   John|
|NULL|   Mike|
|   2|  Alice|
|NULL|Johnson|
+----+-------+

+----+------+
|  id|salary|
+----+------+
|   1|  5000|
|NULL|  3000|
|NULL|  2000|
|NULL|  1500|
+----+------+



In [0]:
df_joined_null_safe = df1.join(df2, df1.id.eqNullSafe(df2.id), how='inner')
df_joined_null_safe.show()

+----+-------+----+------+
|  id|   name|  id|salary|
+----+-------+----+------+
|   1|   John|   1|  5000|
|NULL|   Mike|NULL|  1500|
|NULL|   Mike|NULL|  2000|
|NULL|   Mike|NULL|  3000|
|NULL|Johnson|NULL|  1500|
|NULL|Johnson|NULL|  2000|
|NULL|Johnson|NULL|  3000|
+----+-------+----+------+



I searched and found the null safe join operator in pyspark which is eqNullSafe. Using this the null values are also included in the joined table. TO visualize more clearly, i added 1 more null row in both tables, and as expected, it joins null rows in combination (mxn). Also, it didnot matter which join (left, right, inner) i used, the _null rows_ were the same in count.